# Extract data from ROIs

### Goals of this script
1. load normalized BOLD data (from 01-denoise-sesXX_sub-XXX.ipynb)
2. apply masks
3. save the voxel x TR matrix

### For submitting slurm job:
- approximate time to run for one subject: 1 hr

#### 9/27/22 Modification:
- intersect brain mask and ROI mask to remove any voxels with no signal from the ROI mask

## Define subject

In [ ]:
sub='sub-XXX'

# if using trimmed 'raw' patterns
n_trunc_beginning=9 #Number of volumes to trim/truncate
n_trunc_end=5

version='v4' #for hippocampal masks
binarization_thresh='50' #for hippocampal masks

rsa_ROIs = ['bilateral_oc-temp_v2', 'bilateral_CA2+3+DG_1.5mm', 'left_CA2+3+DG_1.5mm', 'right_CA2+3+DG_1.5mm', 
            'bilateral_CA1_1.5mm', 'left_CA1_1.5mm', 'right_CA1_1.5mm',
            'bilateral_CA2+3_1.5mm', 'left_CA2+3_1.5mm', 'right_CA2+3_1.5mm',
            'bilateral_DG_1.5mm', 'left_DG_1.5mm', 'right_DG_1.5mm']

make_intersected_mask = 0 #1 to intersect brain mask with ROI

## Import necessary packages

In [ ]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np
import nibabel as nib
import nilearn
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.masking import intersect_masks
from nilearn import image
from nilearn import plotting
import scipy.io
from scipy import stats
import os
from os.path import join, exists, split
import pickle 
import time
from pathlib import Path
from shutil import copyfile
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 
%autosave 5

In [ ]:
from platform import python_version
print('The python version is {}.'.format(python_version()))
print('The numpy version is {}.'.format(np.__version__))
print('The nilearn version is {}.'.format(nilearn.__version__))
print('The nibabel version is {}.'.format(nib.__version__))
print('The seaborn version is {}.'.format(sns.__version__))

assert python_version()== '3.7.6'
assert nilearn.__version__=='0.6.2'

## Load settings

In [ ]:
# Set printing precision
np.set_printoptions(precision=2, suppress=True)

# load some helper functions
sys.path.insert(0, '/path/to/code/directory')
import svd_utils
from svd_utils import load_svd_stim_labels, load_svd_epi_data, shift_timing, label2TR

# load some constants
from svd_utils import svd_dir, svd_bids_dir, svd_TR, svd_hrf_lag, run_names, n_runs, TRs_run
shift_size = int(svd_hrf_lag / svd_TR) # Convert the shift into TRs

deriv_dir=svd_bids_dir + 'derivatives/'
anat_dir=deriv_dir + 'deface/'
firstlevel_dir=deriv_dir + 'firstlevel/%s/' % sub

print('ROIs = %s' % (rsa_ROIs))
print('')
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('')


#### Where do you want to save the masked data?

In [ ]:
out_dir = deriv_dir + 'path/to/output'

#### Where are hippocampal masks located? 

In [ ]:
mask_fold_hipp=firstlevel_dir + 'rois_ashs/t1space_%s/threshold-%s/' % (version,binarization_thresh)
print('masking data using masks from %s' %mask_fold_hipp)

#### Where are non-hippocampal masks located? 

In [ ]:
mask_fold=deriv_dir + 'firstlevel/%s/masks/' %sub
print('masking data using masks from %s' %mask_fold)

In [ ]:
# Make a function to load the mask data
def load_svd_mask(ROI_name, sub):
    """Load the mask for the svd data 
    Parameters
    ----------
    ROI_name: string
    sub: string 
    
    Return
    ----------
    the requested mask
    """    
    # load the mask
    maskfile = (mask_fold + sub + "_%s.nii.gz" % (ROI_name))
    mask = nib.load(maskfile)
    print("Loaded %s mask" % (ROI_name))
    return mask

def mask_data(epi_data, mask): 
    """mask the input data with the input mask 
    Parameters
    ----------
    epi_data
    mask
    
    Return
    ----------
    masked data
    """    
    #check that masks and BOLD data match
    assert mask.shape==epi_data.shape[:3] 
    assert mask.header.get_zooms()==epi_data.header.get_zooms()[0:3] #resolution
    assert mask.affine.all()==epi_data.affine.all() #check that affines match
    print('mask shape:', mask.shape, 'dimensions:', mask.header.get_zooms())
    print('mask affine:')
    print(mask.affine)
    
    nifti_masker = NiftiMasker(mask_img=mask)
    epi_masked_data = nifti_masker.fit_transform(epi_data);
    return epi_masked_data

def load_svd_masked_data(directory, subject_name, mask_list):
    masked_data_all = [0] * len(mask_list)

    # Cycle through the masks
    for mask_counter in range(len(mask_list)):
        # load the mask for the corresponding ROI
        this_mask = mask_list[mask_counter]
        mask = load_svd_mask(mask_list[mask_counter], subject_name)
        
        # # plot mask overlayed on subject's T1
        #plot_roi(mask, bg_img=t1_img, title=this_mask)
        
        # mask the data 
        print('extracting masked data for %s' %(this_mask))
        epi_masked_data = mask_data(epi_data, mask)
        epi_masked_data = np.transpose(epi_masked_data)
        
        # Check the dimensionality of the data
        print('voxel by TR matrix - shape: ', epi_masked_data.shape)
        print('')
        
        masked_data_all[mask_counter] = epi_masked_data
        
    return masked_data_all

## Make sure that only voxels included in brain mask are included in oc-temp mask

In [ ]:
# note: ROI here is hardcoded as bilateral_oc-temp
if make_intersected_mask == 1:
    mask_imgs=[]

    brain_mask = mask_fold + sub + '_ses-01_brain.nii.gz'
    roi_mask = mask_fold + sub + '_bilateral_oc-temp.nii.gz'
    brain_img = nib.load(brain_mask)
    roi_img = nib.load(roi_mask)
    print('brain image:', brain_img.shape)
    print(brain_img.affine)
    print('roi:', roi_img.shape)
    print(roi_img.affine)

    mask_imgs.append(brain_mask)
    mask_imgs.append(roi_mask)

    # intersect masks    
    new_mask=intersect_masks(mask_imgs, threshold=1, connected=False)
    print('new mask:', new_mask.shape)
    print(new_mask.affine)
    
    # Save the mask
    output_name = mask_fold + sub + '_bilateral_oc-temp_v2.nii.gz'
    print('Save intersected mask:', output_name)
    print('')

    dimsize=new_mask.header.get_zooms()
    affine_mat = new_mask.affine
    print('Mask dimensions:', dimsize)
    print('')

    hdr = new_mask.header  # get a handle for the .nii file's header
    hdr.set_zooms((dimsize[0], dimsize[1], dimsize[2]))
    nib.save(new_mask, output_name)

## Load fMRI data and apply masks

### LOCALIZER

In [ ]:
execute=0 #1 to run, 0 to skip
ses='ses-00'
task='localizer'
task_index = run_names.index(task)
n_runs_task = n_runs[task_index]
TRs_run_task=TRs_run[task_index]-n_trunc_beginning-n_trunc_end #if data are already trimmed, update TRs_run

print('LIST OF TASKS:', run_names)
print('task index:', task_index)
print('')
print('TR = %s seconds' % (svd_TR))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('')
print('Number of %s runs = %s and TRs per run = %s' % (task, n_runs_task, TRs_run[task_index]))
print('TRs per %s run after trimming = %s' % (task,TRs_run_task))
print('')
print('available ROIs: ', rsa_ROIs)

In [ ]:
if execute==1:
    
    # load normalized BOLD data
    epi_data=[]
    epi_in = (firstlevel_dir  + ses + "/%s_%s_task-%s_run-ALL_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz" % (sub, ses, task, n_trunc_beginning, n_trunc_end))
    epi_data = nib.load(epi_in)
    assert epi_data.shape[3]==n_runs_task*TRs_run_task
    print("Loading data from %s" % (epi_in))
    print('')
    print('epi_data shape: ', epi_data.shape, 'dimensions:', epi_data.header.get_zooms())
    print('epi_data affine:')
    print(epi_data.affine)
    print('')
    
    # Extract voxels for each ROI using NiftiMasker
    masked_data_all = load_svd_masked_data(mask_fold, sub, rsa_ROIs)
    
    # Plot data (first 250 voxels only)
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        plt.figure(figsize=(20,10))
        plt.matshow(masked_data_all[mask_counter][:250,:]) #[voxel,time]
        plt.title(this_mask)
    
    # Save data
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        mat_out = out_dir + '%s_task-%s_run-ALL_space-T1w_trim%dand%dTRs_mask-%s' % (sub, task, n_trunc_beginning, n_trunc_end, this_mask)
        print('saving to file: ', mat_out)
        print('')
        scipy.io.savemat(mat_out, mdict={'data': masked_data_all[mask_counter]})

    print('Saving complete')

else:
    print('Skipping %s task' % (task))     

### STUDY AND POSTSCENES

In [ ]:
execute=0 #1 to run, 0 to skip
ses='ses-01'
task='study'
task2='postScenes'
task_index = run_names.index(task)
task2_index = run_names.index(task2)
n_runs_task = n_runs[task_index]
n_runs_task2 = n_runs[task2_index]
n_runs_total = n_runs_task + n_runs_task2
TRs_run_task=TRs_run[task_index]-n_trunc_beginning-n_trunc_end #if data are already trimmed, update TRs_run
TRs_run_task2=TRs_run[task2_index]-n_trunc_beginning-n_trunc_end

print('LIST OF TASKS:', run_names)
print('task index:', task_index)
print('task index:', task2_index)
print('')
print('TR = %s seconds' % (svd_TR))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('')
print('Number of %s runs = %s and TRs per run = %s' % (task, n_runs_task, TRs_run[task_index]))
print('TRs per %s run after trimming = %s' % (task, TRs_run_task))
print('Number of %s runs = %s and TRs per run = %s' % (task2, n_runs_task2, TRs_run[task2_index]))
print('TRs per %s run after trimming = %s' % (task2, TRs_run_task2))
print('')
print('available ROIs: ', rsa_ROIs)

In [ ]:
if execute==1:
    
    # load normalized BOLD data
    epi_data=[]
    epi_in = (firstlevel_dir  + ses + "/%s_ses-01and02_task-study-and-postscenes_run-ALL_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz" % (sub, n_trunc_beginning, n_trunc_end))
    epi_data = nib.load(epi_in)
    assert epi_data.shape[3]==(n_runs_task*TRs_run_task)+(n_runs_task2*TRs_run_task2)
    print("Loading data from %s" % (epi_in))
    print('')
    print('epi_data shape: ', epi_data.shape, 'dimensions:', epi_data.header.get_zooms())
    print('epi_data affine:')
    print(epi_data.affine)
    print('')
    
    # Extract voxels for each ROI using NiftiMasker
    masked_data_all = load_svd_masked_data(mask_fold, sub, rsa_ROIs)
    
    # Plot data (first 250 voxels only)
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        plt.figure(figsize=(20,10))
        plt.matshow(masked_data_all[mask_counter][:250,:]) #[voxel,time]
        plt.title(this_mask)
    
    # Save data
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        mat_out = out_dir + '%s_task-study-and-postscenes_run-ALL_space-T1w_trim%dand%dTRs_mask-%s' % (sub, n_trunc_beginning, n_trunc_end, this_mask)
        print('saving to file: ', mat_out)
        print('')
        scipy.io.savemat(mat_out, mdict={'data': masked_data_all[mask_counter]})

    print('Saving complete')

else:
    print('Skipping %s task' % ('study-and-postscenes'))

### POSTSCENES

In [ ]:
execute=0 #1 to run, 0 to skip
ses='ses-02'
task='postScenes'
task_index = run_names.index(task)
n_runs_task = n_runs[task_index]
TRs_run_task=TRs_run[task_index]-n_trunc_beginning-n_trunc_end #if data are already trimmed, update TRs_run

print('LIST OF TASKS:', run_names)
print('task index:', task_index)
print('')
print('TR = %s seconds' % (svd_TR))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('')
print('Number of %s runs = %s and TRs per run = %s' % (task, n_runs_task, TRs_run[task_index]))
print('TRs per %s run after trimming = %s' % (task,TRs_run_task))
print('')
print('available ROIs: ', rsa_ROIs)

In [ ]:
if execute==1:
    
    task='postscenes'
    # load normalized BOLD data
    epi_data=[]
    epi_in = (firstlevel_dir  + ses + "/%s_%s_task-%s_run-01_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz" % (sub, ses, task, n_trunc_beginning, n_trunc_end))
    epi_data = nib.load(epi_in)
    assert epi_data.shape[3]==n_runs_task*TRs_run_task
    print("Loading data from %s" % (epi_in))
    print('')
    print('epi_data shape: ', epi_data.shape, 'dimensions:', epi_data.header.get_zooms())
    print('epi_data affine:')
    print(epi_data.affine)
    print('')
    
    # Extract voxels for each ROI using NiftiMasker
    masked_data_all = load_svd_masked_data(mask_fold, sub, rsa_ROIs)
    
    # Plot data (first 250 voxels only)
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        plt.figure(figsize=(20,10))
        plt.matshow(masked_data_all[mask_counter][:250,:]) #[voxel,time]
        plt.title(this_mask)
    
    # Save data
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        mat_out = out_dir + '%s_task-%s_run-01_space-T1w_trim%dand%dTRs_mask-%s' % (sub, task, n_trunc_beginning, n_trunc_end, this_mask)
        print('saving to file: ', mat_out)
        print('')
        scipy.io.savemat(mat_out, mdict={'data': masked_data_all[mask_counter]})

    print('Saving complete')

else:
    print('Skipping %s task' % (task))    

### POSTFACES

In [ ]:
execute=0 #1 to run, 0 to skip
ses='ses-02'
task='postFaces'
task_index = run_names.index(task)
n_runs_task = n_runs[task_index]
TRs_run_task=TRs_run[task_index]-n_trunc_beginning-n_trunc_end #if data are already trimmed, update TRs_run

print('LIST OF TASKS:', run_names)
print('task index:', task_index)
print('')
print('TR = %s seconds' % (svd_TR))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('')
print('Number of %s runs = %s and TRs per run = %s' % (task, n_runs_task, TRs_run[task_index]))
print('TRs per %s run after trimming = %s' % (task,TRs_run_task))
print('')
print('available ROIs: ', rsa_ROIs)

In [ ]:
if execute==1:
    
    task='postfaces'
    # load normalized BOLD data
    epi_data=[]
    epi_in = (firstlevel_dir  + ses + "/%s_%s_task-%s_run-01_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz" % (sub, ses, task, n_trunc_beginning, n_trunc_end))
    epi_data = nib.load(epi_in)
    assert epi_data.shape[3]==n_runs_task*TRs_run_task
    print("Loading data from %s" % (epi_in))
    print('')
    print('epi_data shape: ', epi_data.shape, 'dimensions:', epi_data.header.get_zooms())
    print('epi_data affine:')
    print(epi_data.affine)
    print('')
    
    # Extract voxels for each ROI using NiftiMasker
    masked_data_all = load_svd_masked_data(mask_fold, sub, rsa_ROIs)
    
    # Plot data (first 250 voxels only)
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        plt.figure(figsize=(20,10))
        plt.matshow(masked_data_all[mask_counter][:250,:]) #[voxel,time]
        plt.title(this_mask)
    
    # Save data
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        mat_out = out_dir + '%s_task-%s_run-01_space-T1w_trim%dand%dTRs_mask-%s' % (sub, task, n_trunc_beginning, n_trunc_end, this_mask)
        print('saving to file: ', mat_out)
        print('')
        scipy.io.savemat(mat_out, mdict={'data': masked_data_all[mask_counter]})

    print('Saving complete')

else:
    print('Skipping %s task' % (task))    

### REWARD

In [ ]:
execute=0 #1 to run, 0 to skip
ses='ses-02'
task='reward'
task_index = run_names.index(task)
n_runs_task = n_runs[task_index]
TRs_run_task=TRs_run[task_index]-n_trunc_beginning-n_trunc_end #if data are already trimmed, update TRs_run

print('LIST OF TASKS:', run_names)
print('task index:', task_index)
print('')
print('TR = %s seconds' % (svd_TR))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('')
print('Number of %s runs = %s and TRs per run = %s' % (task, n_runs_task, TRs_run[task_index]))
print('TRs per %s run after trimming = %s' % (task,TRs_run_task))
print('')
print('available ROIs: ', rsa_ROIs)

In [ ]:
if execute==1:
    
    # load normalized BOLD data
    epi_data=[]
    epi_in = (firstlevel_dir  + ses + "/%s_%s_task-%s_run-ALL_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz" % (sub, ses, task, n_trunc_beginning, n_trunc_end))
    epi_data = nib.load(epi_in)
    assert epi_data.shape[3]==n_runs_task*TRs_run_task
    print("Loading data from %s" % (epi_in))
    print('')
    print('epi_data shape: ', epi_data.shape, 'dimensions:', epi_data.header.get_zooms())
    print('epi_data affine:')
    print(epi_data.affine)
    print('')
    
    # Extract voxels for each ROI using NiftiMasker
    masked_data_all = load_svd_masked_data(mask_fold, sub, rsa_ROIs)
    
    # Plot data (first 250 voxels only)
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        plt.figure(figsize=(20,10))
        plt.matshow(masked_data_all[mask_counter][:250,:]) #[voxel,time]
        plt.title(this_mask)
    
    # Save data
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        mat_out = out_dir + '%s_task-%s_run-ALL_space-T1w_trim%dand%dTRs_mask-%s' % (sub, task, n_trunc_beginning, n_trunc_end, this_mask)
        print('saving to file: ', mat_out)
        print('')
        scipy.io.savemat(mat_out, mdict={'data': masked_data_all[mask_counter]})

    print('Saving complete')

else:
    print('Skipping %s task' % (task))    

### FAMILIARIZATION

In [ ]:
execute=0 #1 to run, 0 to skip
ses='ses-02'
task='familiarization'
task_index = run_names.index(task)
n_runs_task = n_runs[task_index]
TRs_run_task=TRs_run[task_index]-n_trunc_beginning-n_trunc_end #if data are already trimmed, update TRs_run

print('LIST OF TASKS:', run_names)
print('task index:', task_index)
print('')
print('TR = %s seconds' % (svd_TR))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('')
print('Number of %s runs = %s and TRs per run = %s' % (task, n_runs_task, TRs_run[task_index]))
print('TRs per %s run after trimming = %s' % (task,TRs_run_task))
print('')
print('available ROIs: ', rsa_ROIs)

In [ ]:
if execute==1:  
    
    # load normalized BOLD data
    epi_data=[]
    epi_in = (firstlevel_dir  + ses + "/%s_%s_task-%s_run-ALL_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz" % (sub, ses, task, n_trunc_beginning, n_trunc_end))
    epi_data = nib.load(epi_in)
    assert epi_data.shape[3]==n_runs_task*TRs_run_task
    print("Loading data from %s" % (epi_in))
    print('')
    print('epi_data shape: ', epi_data.shape, 'dimensions:', epi_data.header.get_zooms())
    print('epi_data affine:')
    print(epi_data.affine)
    print('')
    
    # Extract voxels for each ROI using NiftiMasker
    masked_data_all = load_svd_masked_data(mask_fold, sub, rsa_ROIs)
    
    # Plot data (first 250 voxels only)
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        plt.figure(figsize=(20,10))
        plt.matshow(masked_data_all[mask_counter][:250,:]) #[voxel,time]
        plt.title(this_mask)
    
    # Save data
    for mask_counter in range(len(rsa_ROIs)):
        this_mask = rsa_ROIs[mask_counter]
        mat_out = out_dir + '%s_task-%s_run-ALL_space-T1w_trim%dand%dTRs_mask-%s' % (sub, task, n_trunc_beginning, n_trunc_end, this_mask)
        print('saving to file: ', mat_out)
        print('')
        scipy.io.savemat(mat_out, mdict={'data': masked_data_all[mask_counter]})

    print('Saving complete')

else:
    print('Skipping %s task' % (task))    